In [17]:
import cv2
from ultralytics import YOLO
import torchvision.ops as ops
import colorsys
import datetime

In [18]:
checkpoint = YOLO('../ml_models/model2_best.pt')
# INPUT = 0 # webcam
INPUT = "../data/20230307_143020.mp4"  # video

In [19]:

def float_to_color(f):
    """
    Convert a float from 0 to 1 to a BGR color from red to yellow to green.
    """
    hue = 0.33 * f
    r, g, b = colorsys.hsv_to_rgb(hue, 1, 1)
    r = int(r * 255)
    g = int(g * 255)
    b = int(b * 255)
    return (b, g, r)


In [20]:

def nms_analysis(image, model):
    threshold = 0
    result = model(image)
    boxes = result[0].boxes.xyxy.cuda()
    scores = result[0].boxes.conf.cuda()
    print(f"Found {len(boxes)} boxes and {len(scores)} scores")
    # Non-Maximum Suppression
    indices = ops.nms(boxes, scores, threshold)

    cv2.putText(image, f"Paddles: {len(indices)}", (15, 30), cv2.FONT_HERSHEY_SIMPLEX, .75,
                (255, 255, 255), 2)

    if indices.numel() == 0:
        return image

    boxes = boxes[indices]
    scores = scores[indices]
    for box, score in zip(boxes, scores):
        if score >= threshold:
            box = [int(i) for i in box]
            cv2.rectangle(image, (box[0], box[1]), (box[2], box[3]), float_to_color(score), 2)
            cv2.putText(image, f"Score: {round(float(score), 2)}", (box[0], box[3] + 36), cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (255, 255, 255), 2)

    return image



In [21]:
import time


def analyse_video():
    cap = cv2.VideoCapture(INPUT)
    prev_timestamp = time.time()

    while cap.isOpened():
        ret, frame = cap.read()
        img = nms_analysis(frame, checkpoint)
        current_timestamp = time.time()
        time_delta = current_timestamp - prev_timestamp
        cv2.putText(img, f"FPS: {int(1 / time_delta)} delta: {round(time_delta * 1000, 2)}ms", (15, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, .75,
                    (255, 255, 255), 2)
        if img is None or img.shape[0] == 0 or img.shape[1] == 0:
            break

        cv2.imshow('image', img)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        prev_timestamp = current_timestamp

    cap.release()
    cv2.destroyAllWindows()


analyse_video()


0: 480x800 (no detections), 13.1ms
Speed: 1.0ms preprocess, 13.1ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 10.5ms
Speed: 1.0ms preprocess, 10.5ms inference, 1.5ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 10.5ms
Speed: 0.0ms preprocess, 10.5ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 10.2ms
Speed: 1.1ms preprocess, 10.2ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 11.5ms
Speed: 1.0ms preprocess, 11.5ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 9.1ms


Found 0 boxes and 0 scores
Found 0 boxes and 0 scores
Found 0 boxes and 0 scores
Found 0 boxes and 0 scores
Found 0 boxes and 0 scores


Speed: 1.0ms preprocess, 9.1ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 8.0ms
Speed: 1.5ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 10.5ms
Speed: 1.0ms preprocess, 10.5ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 10.1ms
Speed: 1.0ms preprocess, 10.1ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 11.5ms
Speed: 1.0ms preprocess, 11.5ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 11.5ms
Speed: 0.0ms preprocess, 11.5ms inference, 1.1ms postprocess per image at shape (1, 3, 800, 800)



Found 0 boxes and 0 scores
Found 0 boxes and 0 scores
Found 0 boxes and 0 scores
Found 0 boxes and 0 scores
Found 0 boxes and 0 scores
Found 0 boxes and 0 scores
Found 0 boxes and 0 scores


0: 480x800 (no detections), 9.5ms
Speed: 1.0ms preprocess, 9.5ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 10.5ms
Speed: 0.0ms preprocess, 10.5ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 8.5ms
Speed: 1.0ms preprocess, 8.5ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 10.0ms
Speed: 0.0ms preprocess, 10.0ms inference, 1.5ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 9.5ms
Speed: 0.0ms preprocess, 9.5ms inference, 2.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 8.5ms
Speed: 1.0ms preprocess, 8.5ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 9.0ms
Speed: 0.0ms preprocess, 9.0ms inference, 0.0ms postprocess per image at shape (1, 3, 800, 800)



...

Found 0 boxes and 0 scores
Found 0 boxes and 0 scores
Found 0 boxes and 0 scores
Found 0 boxes and 0 scores
Found 0 boxes and 0 scores
Found 0 boxes and 0 scores
Found 0 boxes and 0 scores
Found 0 boxes and 0 scores


Speed: 1.0ms preprocess, 8.8ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 6.5ms
Speed: 1.0ms preprocess, 6.5ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 8.5ms
Speed: 0.0ms preprocess, 8.5ms inference, 0.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 9.5ms
Speed: 0.0ms preprocess, 9.5ms inference, 0.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 9.7ms
Speed: 0.0ms preprocess, 9.7ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 8.5ms
Speed: 0.0ms preprocess, 8.5ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 9.0ms
Speed: 0.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)



Found 0 boxes and 0 scores
Found 0 boxes and 0 scores
Found 0 boxes and 0 scores
Found 0 boxes and 0 scores
Found 0 boxes and 0 scores
Found 0 boxes and 0 scores
Found 0 boxes and 0 scores


0: 480x800 (no detections), 8.5ms
Speed: 1.0ms preprocess, 8.5ms inference, 0.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 7.0ms
Speed: 1.0ms preprocess, 7.0ms inference, 0.0ms postprocess per image at shape (1, 3, 800, 800)


Found 0 boxes and 0 scores
Found 0 boxes and 0 scores
